# Web Scraping Notebook

This notebook is set up for web scraping tasks.


In [1]:
# Import common libraries
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import seaborn as sns

print("Libraries imported successfully!")


Libraries imported successfully!
